In [1]:

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc

In [2]:
from sklearn.datasets import load_digits

In [4]:
data, labels = load_digits(return_X_y=True)
(n_samples, n_features) = data.shape
n_digits = np.unique(labels).size
print(f"# digits: {n_digits}; # samples: {n_samples}; # features {n_features}")

# digits: 10; # samples: 1797; # features 64
